<a href="https://colab.research.google.com/github/rinky88-ai/Phi-3-SLM-Q-A/blob/main/Ph3_slm_Spiritual_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.40.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.


In [ ]:
!pip install huggingface_hub peft bitsandbytes
!pip install trl xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 136.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2


In [ ]:
!pip install torch

In [ ]:
import json
def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

train = load_jsonl("phi3_sft_spiritual_train.jsonl")
val = load_jsonl("phi3_sft_spiritual_validation.jsonl")
test = load_jsonl("phi3_sft_spiritual_test.jsonl")
print(f"Train: {len(train)} | Val: {len(val)} | Test: {len(test)}")
print('\nSample:')
import random
print(random.choice(train))

Train: 150 | Val: 20 | Test: 30

Sample:
{'instruction': 'Answer the question based on the provided context.', 'input': 'Title: Rituals\nContext: Rituals — an important aspect of spiritual life related to rituals. Practicing rituals often leads to inner growth and well-being..\nQuestion: What is Rituals?', 'output': 'an important aspect of spiritual life related to rituals.'}


In [ ]:
from datasets import Dataset, DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_list(train),
    "validation": Dataset.from_list(val),
    "test": Dataset.from_list(test)
})
print(dataset)

def format_fn(example):
    if example.get("input"):
        prompt = f"Instruction: {example['instruction']}\nInput: {example['input']}\nResponse:"
    else:
        prompt = f"Instruction: {example['instruction']}\nResponse:"
    return {"prompt": prompt, "label": example["output"]}

dataset = dataset.map(lambda ex: format_fn(ex), remove_columns=dataset["train"].column_names)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 20
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 30
    })
})


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

{'prompt': 'Instruction: Answer the question based on the provided context.\nInput: Title: Nyaya\nContext: Nyaya — an important aspect of spiritual life related to nyaya. Practicing nyaya often leads to inner growth and well-being..\nQuestion: What is Nyaya?\nResponse:', 'label': 'an important aspect of spiritual life related to nyaya.'}


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "microsoft/Phi-3-mini-4k-instruct"  # change if needed
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto", load_in_8bit=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
print('Model and tokenizer ready')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model and tokenizer ready


In [ ]:
max_length = 1024
def tokenize_examples(batch):
    prompts = batch["prompt"]
    labels = batch["label"]
    tokenized = tokenizer(prompts, truncation=True, padding="max_length", max_length=max_length)
    with tokenizer.as_target_tokenizer():
        tokenized_targets = tokenizer(labels, truncation=True, padding="max_length", max_length=max_length)
    tokenized["labels"] = tokenized_targets["input_ids"]
    return tokenized

tokenized = dataset.map(tokenize_examples, batched=True, remove_columns=dataset["train"].column_names)
print(tokenized)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 30
    })
})


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["qkv_proj", "o_proj"], # Updated target modules for Phi-3
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
print('PEFT model prepared')

PEFT model prepared


In [ ]:
training_args = TrainingArguments(
    output_dir="./phi3_diwali_lora",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=data_collator,
)

# Start training:
trainer.train()
print('Trainer ready. Uncomment trainer.train() to start training.')

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rinky88 (rinky88-intel) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,14.511100
20,12.313400
30,10.080800
40,7.733100
50,5.329300
60,3.183100
70,0.822600
80,0.266500
90,0.175000
100,0.134100


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen

Trainer ready. Uncomment trainer.train() to start training.


In [ ]:
prompt = "What is Karma?"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device) # Move input_ids to the same device as the model
output_ids = model.generate(input_ids, max_length=100, num_return_sequences=1,use_cache=False)
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_text)

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Why people light Lamps and Dia in Diwali?.


### Answer:

Diwali, also known as the Festival of Lights, is one of the most significant festivals in Hindu culture, celebrated by millions of people across the world. The tradition of lighting lamps and candles during Diwali has deep historical, religious, and cultural roots. Here are some reasons why people light lamps and candles during this festival:

1.


In [ ]:
# Save adapters
adapter_dir = "./diwali_lora_adapter"
model.save_pretrained(adapter_dir)
print('Saved adapters to', adapter_dir)

# Quick inference (uses the model currently loaded)
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer,use_cache=False, max_length=256)

def answer_question(question, context=""):
    if context:
        prompt = f"Instruction: {question}\nInput: {context}\nResponse:"
    else:
        prompt = f"Instruction: {question}\nResponse:"
    return generator(prompt, do_sample=False, num_return_sequences=1)[0]['generated_text']

q = "What is Karma?"
print('Prompt:', q)
print('Answer:', answer_question(q))

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Saved adapters to ./diwali_lora_adapter
Prompt: What is Karma?


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Answer: Instruction: What is Karma?
Response: Karma is a concept found in several Eastern religions, including Hinduism, Buddhism, and Jainism. It refers to the principle of cause and effect, where the intent and actions of an individual influence their future. Karma is often understood as the moral law of the universe, where good actions lead to positive outcomes, and bad actions lead to negative consequences.

In Hinduism, karma is closely tied to the cycle of birth, death, and rebirth, known as samsara. It is believed that the accumulated karma from past lives determines the circumstances of one'
